In [2]:
pip install textdistance

In [3]:
import pandas_profiling


In [4]:
import pandas as pd
import numpy as np
import textdistance
import re
from collections import Counter

In [5]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link =  "https://drive.google.com/open?id=1ziJ_81tOLm_Krclc5sBxKPJ1yc4wTQzo" #LINK DA BASE ORIGINAL
#"https://drive.google.com/file/d/10Un2f-CEL3D4oPZbP6nlJhtt8wAnUHn-/view?usp=sharing" #LINK DA BASE USADA NO TCC
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('book.txt')  



1ziJ_81tOLm_Krclc5sBxKPJ1yc4wTQzo


In [6]:
words = []
with open('book.txt', 'r') as f:
    file_name_data = f.read()
    file_name_data=file_name_data.lower()
    words = re.findall('\w+',file_name_data)
# This is our vocabulary
V = set(words)
print(f"The first ten words in the text are: \n{words[0:10]}")
print(f"There are {len(V)} unique words in the vocabulary.")

The first ten words in the text are: 
['the', 'project', 'gutenberg', 'ebook', 'of', 'moby', 'dick', 'or', 'the', 'whale']
There are 17647 unique words in the vocabulary.


In [7]:
word_freq_dict = {}  
word_freq_dict = Counter(words)
print(word_freq_dict.most_common()[0:30])

[('the', 14703), ('of', 6742), ('and', 6517), ('a', 4799), ('to', 4707), ('in', 4238), ('that', 3081), ('it', 2534), ('his', 2530), ('i', 2120), ('he', 1896), ('but', 1823), ('s', 1819), ('with', 1769), ('as', 1752), ('is', 1751), ('was', 1646), ('for', 1644), ('all', 1544), ('this', 1439), ('at', 1335), ('whale', 1230), ('by', 1222), ('not', 1173), ('from', 1104), ('on', 1073), ('so', 1066), ('him', 1065), ('be', 1063), ('you', 958)]


In [8]:
df = pd.DataFrame(words, columns=['Words'])
df.head()

,Words
0,the
1,project
2,gutenberg
3,ebook
4,of


In [9]:
probs = {}     
Total = sum(word_freq_dict.values())    
for k in word_freq_dict.keys():
    probs[k] = word_freq_dict[k]/Total

In [15]:
def my_autocorrect(input_word):
    input_word = input_word.lower()
    if input_word in V:
      return 'Your word seems to be correct'
    else:
        similarities = [1-(textdistance.Jaccard(qval=2).distance(v,input_word)) for v in word_freq_dict.keys()]
        
        df = pd.DataFrame.from_dict(probs, orient='index').reset_index()
        df = df.rename(columns={'index':'Word', 0:'Prob'})
        df['Similarity'] = similarities
        output = df.sort_values(['Similarity', 'Prob'], ascending=False).head()
        return(output)

In [35]:
my_autocorrect('nelvilie')

,Word,Prob,Similarity
11,melville,0.000018,0.400000
5797,belie,0.000009,0.375000
9058,relieving,0.000004,0.363636
11703,believing,0.000004,0.363636
4541,relief,0.000027,0.333333
